In [11]:
import requests
import json
import time
import pandas as pd
import numpy as np
from WebCrawlerLibrary import gen_query_url, parse_response, extract_data_from_api, query_start_date
from datetime import datetime, timedelta

In [12]:
# Design dataframe
API_COLUMNS = ["Title", "Description", "NewsURL", "PublishedTime", "SourceName" , "SourceURL"]

api_content = pd.read_csv("../Data/NewsContent.csv", index_col=[0])
# api_content = pd.read_csv("../Data/NewsContent.csv")

In [13]:
api_content_current_session = pd.DataFrame(columns=API_COLUMNS)

In [14]:
# query_index_date.strftime(datetime_format) != datetime.now().strftime(datetime_format)

In [15]:
# Process query days pipeline
datetime_format = "%Y-%m-%d"
# start_date = datetime(2020,1,29)
start_date = query_start_date(api_content)
# end_date =  datetime(2020,2,1)

query_index_date = start_date 

status_code = 200
counter = 0
max_count = 49
first_time = False


while counter < max_count and status_code == 200 and query_index_date.strftime(datetime_format) != datetime.now().strftime(datetime_format):
    # counter does not surpass 100 queryies per day
    # status_code is still successful
    # index date has yet to surpass today
    
    # Generate URLs
    query_index_date_next = query_index_date + timedelta(days=1)
    url = gen_query_url(query_index_date.strftime(datetime_format), query_index_date_next.strftime(datetime_format) )
    
    # Parse URLs
    response = requests.get(url)
    
    # get Status_code and content
    status_code, output_content = parse_response(response)
    
    # Parse content
    if status_code == 200 and output_content != "no returns":    
        new_api_data = extract_data_from_api(output_content)
    
    if counter == 0 and first_time == True:
        api_content = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(api_content)
    else :
        new_api_df = pd.DataFrame(new_api_data, columns = API_COLUMNS)
#         print(new_api_df)
        api_content = api_content.append(new_api_df, ignore_index = True).drop_duplicates()
        api_content_current_session = api_content_current_session.append(new_api_df, ignore_index = True).drop_duplicates()
    

    # Post Processing conditions
    query_index_date = query_index_date_next
    counter += 1
    if counter == max_count:
        print("Has Reached Max Count of " + str(counter))
    if status_code != 200:
        print("Status Code " + str(status_code))
    time.sleep(5)

https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-08-14&maxdate=2020-08-15&in=title
https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-08-15&maxdate=2020-08-16&in=title
https://gnews.io/api/v3/search?q=covid19|coronavirus us&token=837a1cfd8cff3793dca10c2b9478dcce&lang=en&country=us&mindate=2020-08-16&maxdate=2020-08-17&in=title


In [16]:
# temp = api_content_current_session.copy()
# temp = temp.drop_duplicates(["Title" , "PublishedTime"])
# api_content = api_content.append(temp, ignore_index = True).drop_duplicates()


In [17]:
api_content.PublishedTime   = pd.to_datetime(api_content.PublishedTime, utc=True).dt.strftime('%Y-%m-%d')
api_content_current_session.PublishedTime   = pd.to_datetime(api_content_current_session.PublishedTime, utc=True).dt.strftime('%Y-%m-%d')

api_content = api_content.drop_duplicates(["Title", "PublishedTime"]).sort_values(["PublishedTime"])
api_content

,Title,Description,NewsURL,PublishedTime,SourceName,SourceURL
0,White House doesn't rule out suspending US-Chi...,The White House is not ruling out suspending f...,https://abcnews.go.com/Politics/white-house-co...,2020-01-29,ABC,https://abcnews.go.com
1,Coronavirus Crisis In China To Have Economic I...,“China controls almost 100% of the generic med...,https://dfw.cbslocal.com/2020/01/29/coronaviru...,2020-01-29,CBS Boston / WBZ,https://dfw.cbslocal.com
2,Coronavirus cases in US should not cause mass ...,A deadly coronavirus that's killed at least 13...,https://www.foxnews.com/health/coronavirus-cas...,2020-01-29,Fox News,https://www.foxnews.com
3,US learned from Ebola but is 'nowhere near as ...,"“The Department of Homeland Security said, ‘Ac...",https://www.usatoday.com/story/news/health/202...,2020-01-29,USA Today,https://www.usatoday.com
4,Coronavirus: Japan and US fly citizens home as...,Japan and US airlift hundreds out of Wuhan as ...,https://www.theguardian.com/world/2020/jan/29/...,2020-01-29,The Guardian,https://www.theguardian.com
...,...,...,...,...,...,...
1692,'This Is Us' to Tackle Coronavirus in Season 5,Creator Dan Fogelman revealed the news on Sund...,https://www.msn.com/en-us/tv/news/this-is-us-t...,2020-08-16,Entertainment Tonight on MSN.com,https://www.msn.com
1695,Coronavirus latest: US military reinstates res...,New Zealand postpones its general election for...,https://www.ft.com/content/4a8cff4d-ebfc-335f-...,2020-08-16,The Financial Times,https://www.ft.com
1696,"US-Canada, Mexico border restrictions extended...",U.S. and Mexico border will remain closed to n...,https://www.usatoday.com/story/travel/news/202...,2020-08-16,USA Today,https://www.usatoday.com
1697,US coronavirus: Saliva-based Covid-19 test cou...,"""If cheap alternatives like SalivaDirect can b...",https://www.msn.com/en-us/news/us/us-coronavir...,2020-08-16,CNN on MSN.com,https://www.msn.com


In [18]:
api_content_current_session.to_csv("../Data/NewContentCurrentSession.csv")
api_content.to_csv("../Data/NewsContent.csv")

---

In [19]:

# import GetOldTweets3 as gt

https://ieee-dataport.org/open-access/coronavirus-covid-19-tweets-dataset

In [20]:
# temp1 = temp_cur_session.drop_duplicates(["Title","PublishedTime"])
# api_content_current_session = temp1
# temp_cur_session